<a href="https://colab.research.google.com/github/tesseract314/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/module1-Intro-to-Neural-Networks/LS_DS_431_Intro_to_NN_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to Neural Networks Assignment

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
Inputs are information from the outside world that are given to the neural network. All inputs make up the Input Layer.

### Hidden Layer:
The Hidden Layer is made up of many hidden nodes, which have no connection to the outside world. Their purpose is to take information from the Input Layer, do caclulations on that information and pass the resulting information to the Output Layer.

### Output Layer:
The Output Layer collect information from the Hidden Layer, does calculation and transfers information from the neural network to the outside world.

### Neuron:
A Neuron is a single and basic unit of computation in a neural network, also referred to as a node. It recieves information from the outside world or another Neuron and computes an output.

### Weight:
A Weight is given to each input and is assigned on the basis of its relative importance to other inputs.

### Activation Function:
The Activation Function is a mathematical function that introduces non-linearity to the outputs of neurons.

### Node Map:
The Node Map shows the composition of the nodes within a neural network, including the Input Layer, Hidden Layer(s) and Output Layer. It often shows the weights associated with the inputs as well.

### Perceptron:
The Perceptron is a linear binary classification algorithm used to calculate outputs in a neural network.



## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

inputs-->weighted sum of inputs-->activattion output(with some kind of function)-->get error-->adjustments to weights-->add bias-->repeat all except inputs to get to the output of least error

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [61]:
# make dataframe
import pandas as pd
import numpy as np

data = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
    [1, 1, 0]
])

df = pd.DataFrame(data, columns=['x1', 'x2', 'y'])
df.head()

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [62]:
# if both inputs are true, the output will be 0
nand_gate = []
for i, ii in zip(df['x1'].tolist(), df['x2'].tolist()):
  if (i==1) & (ii==1):
    nand_gate.append(0)
  else:
    nand_gate.append(1)
    
nand_gate

[1, 1, 1, 0]

In [63]:
# add nand gate results to df
df['nand_gate'] = nand_gate
df.head()

,x1,x2,y,nand_gate
0,0,0,1,1
1,1,0,1,1
2,0,1,1,1
3,1,1,0,0


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [52]:
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
df.shape

(768, 9)

In [9]:
df.corr()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341,0.221898
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514,0.466581
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528,0.065068
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970,0.074752
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163,0.130548
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242,0.292695
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561,0.173844
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000,0.238356
Outcome,0.221898,0.466581,0.065068,0.074752,0.130548,0.292695,0.173844,0.238356,1.000000


In [0]:
# tutorial from https://medium.com/@thomascountz/19-line-line-by-line-python-perceptron-b6f113b161f3
# perceptron class
class Perceptron(object): # create new perceptron class
  
  # init method
  def __init__(self, no_of_inputs, niters=100, learning_rate=0.01): # setting some params e.g. number of inputs, number of iterations (epochs) and learning rate
    self.niters = niters # set niters to instance variable
    self.learning_rate = learning_rate # set learning rate to instance variable
    self.weights = np.zeros(no_of_inputs + 1) # setting all weights equal to zero, the +1 is the bias weight
    
  # predict method
  def predict(self, inputs): # this method houses the algorithm (activation function) -- takes 'inputs'as an argument
    summation = np.dot(inputs, self.weights[1:]) + self.weights[0] # dot product of inputs and weights
    # if summation greater than 0, make activation 1
    if summation > 0:
      activation = 1
    # else make activation 0
    else:
      activation = 0
    return activation # return activation
  
  # train method -- takes two arguments, training inputs and labels
  def train(self, training_inputs, labels):
    for _ in range(self.niters): # for loop will run for all niters
      for inputs, label in zip(training_inputs, labels): # zip training_inputs and labels together to create iterable object
        prediction =  self.predict(inputs) # predict each training input and store in predictions variable
        self.weights[1:] += self.learning_rate * (label - prediction) * inputs # adjust weights
        self.weights[0] += self.learning_rate * (label - prediction) # adjust bias but don't multiply by inputs

In [54]:
training_inputs = df.drop(columns='Outcome').as_matrix()
training_inputs

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [55]:
labels = df['Outcome'].as_matrix()
labels

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,

In [56]:
perceptron = Perceptron(df.drop(columns='Outcome').shape[1], niters=10000, learning_rate=0.001)
perceptron

In [0]:
perceptron.train(training_inputs, labels)

In [58]:
preds = []
for input_ in training_inputs:
  pred = perceptron.predict(input_)
  preds.append(pred)
len(preds) == df.shape[0]

True

In [59]:
df['perceptron_preds'] = preds
df['pred_correct'] = df['perceptron_preds'] == df['Outcome']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,perceptron_preds,pred_correct
0,6,148,72,35,0,33.6,0.627,50,1,1,True
1,1,85,66,29,0,26.6,0.351,31,0,0,True
2,8,183,64,0,0,23.3,0.672,32,1,1,True
3,1,89,66,23,94,28.1,0.167,21,0,0,True
4,0,137,40,35,168,43.1,2.288,33,1,1,True


In [60]:
df['pred_correct'].value_counts(normalize=True)

True     0.682292
False    0.317708
Name: pred_correct, dtype: float64

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?